In [1]:
import numpy as np
import pandas as pd

import os
#os.sys.path
import sys
sys.path.append('../src')

from pathlib import Path

import matplotlib.pyplot as plt
from PIL import Image
from numpy import asarray
import cv2

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
# importing tensorflow model libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import model_from_json,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K
import json
import time

In [3]:
from sklearn.model_selection import train_test_split

### IMGS

In [4]:
inp1 = Path.home()/'Iron'/'inp1'

In [5]:
train_imgs = inp1/'TRAIN'
train_imgs_haar = inp1/'TRAIN_haar'
val_imgs = inp1/'VALIDATION'
test_imgs = inp1/'TEST'
demo_imgs = Path.cwd().parent/'demo'
demo_imgs_faces = Path.cwd().parent/'demo_haar'
demo_imgs_haar = Path.cwd().parent/'demo_faces'

### DATASETS to check

In [41]:
# df_fer_ok --> sin dummies
# df_fer_top --> con dummies!

In [6]:
df_fer = pd.read_csv(inp1/'Fer.csv',encoding = "ISO-8859-1")
df_train = pd.read_csv(inp1/'Training_Data.csv',encoding = "ISO-8859-1")
df_test = pd.read_csv(inp1/'Testing_Data.csv',encoding = "ISO-8859-1")
df_val = pd.read_csv(inp1/'Validation_Data.csv',encoding = "ISO-8859-1")

In [7]:
face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')

In [8]:
df = df_fer.copy()
print(df.shape, df.size, df.columns)

(35887, 3) 107661 Index(['emotion', 'pixels', 'Usage'], dtype='object')


In [9]:
df.emotion.unique()

array([0, 2, 4, 6, 3, 5, 1])

In [10]:
emos = {0:'Angry',1: 'Disgust',2:'Fear',3:'Happy',4:'Sad',5:'Surprise',6:'Neutral'}
#df['emos'] = df.emotion.map(emos)
df['emotion_names'] = df.emotion.map(emos)

In [11]:
emos2 = {0:'unhappy', 1:'unhappy',2:'unhappy',3:'happy',4:'unhappy',5:'unhappy',6:'unhappy'}

In [12]:
df['emo'] = df.emotion.map(emos2).to_numpy()

In [13]:
df.head()

,emotion,pixels,Usage,emotion_names,emo
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry,unhappy
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry,unhappy
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear,unhappy
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad,unhappy
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral,unhappy


In [14]:
emo = pd.get_dummies(df['emo']).to_numpy()

In [15]:
df2 = pd.get_dummies(df['emotion']).to_numpy()

In [17]:
df

,emotion,pixels,Usage,emotion_names,emo
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry,unhappy
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry,unhappy
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear,unhappy
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad,unhappy
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral,unhappy
...,...,...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest,Neutral,unhappy
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest,Happy,happy
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest,Angry,unhappy
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest,Happy,happy


In [18]:
df.columns

Index(['emotion', 'pixels', 'Usage', 'emotion_names', 'emo'], dtype='object')

In [19]:
drpp = ['emotion', 'Usage']
df2 = df.drop(drpp, axis=1)

In [20]:
pth1 = Path.cwd().parent.parent

In [21]:
pth2 = Path.home()/'Iron'/'data_processed'

In [22]:
df2.to_csv(str(pth2/'df_fer_a.csv'))

In [23]:
df3 = df2.copy()

In [24]:
df3['pixar1'] = [[float(x) for x in each.split()] for each in df3['pixels']]
df3['pixar2'] = df3['pixar1'].apply(lambda x: np.asarray(x).reshape(48,48)).apply(lambda x:x.astype('float32'))

In [25]:
df3['emo_arr1'] = df3.pixar2.copy()

In [26]:
df3['emo_arr'] = df3['emo_arr1'].apply(lambda x: np.array([[[c] for c in i] for i in x])) 

In [28]:
df5 = df3.copy()

In [29]:
df5.head(2)

,pixels,emotion_names,emo,pixar1,pixar2,emo_arr1,emo_arr
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Angry,unhappy,"[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63....","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[[70.0], [80.0], [82.0], [72.0], [58.0], [58...."
1,151 150 147 155 148 133 111 140 170 174 182 15...,Angry,unhappy,"[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 111...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[[151.0], [150.0], [147.0], [155.0], [148.0],..."


In [31]:
df5.to_csv(pth2/'df_fer_ok.csv')


In [32]:
df5.head()

,pixels,emotion_names,emo,pixar1,pixar2,emo_arr1,emo_arr
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Angry,unhappy,"[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63....","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[[70.0], [80.0], [82.0], [72.0], [58.0], [58...."
1,151 150 147 155 148 133 111 140 170 174 182 15...,Angry,unhappy,"[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 111...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[[151.0], [150.0], [147.0], [155.0], [148.0],..."
2,231 212 156 164 174 138 161 173 182 200 106 38...,Fear,unhappy,"[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 161...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16...","[[[231.0], [212.0], [156.0], [164.0], [174.0],..."
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Sad,unhappy,"[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20....","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20...","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20...","[[[24.0], [32.0], [36.0], [30.0], [32.0], [23...."
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Neutral,unhappy,"[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[4.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0..."


In [37]:
df6 = pd.get_dummies(df5, columns=['emo'])


In [38]:
df6.head()

,pixels,emotion_names,pixar1,pixar2,emo_arr1,emo_arr,emo_happy,emo_unhappy
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Angry,"[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63....","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[[70.0], [80.0], [82.0], [72.0], [58.0], [58....",0,1
1,151 150 147 155 148 133 111 140 170 174 182 15...,Angry,"[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 111...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[[151.0], [150.0], [147.0], [155.0], [148.0],...",0,1
2,231 212 156 164 174 138 161 173 182 200 106 38...,Fear,"[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 161...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16...","[[[231.0], [212.0], [156.0], [164.0], [174.0],...",0,1
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Sad,"[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20....","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20...","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20...","[[[24.0], [32.0], [36.0], [30.0], [32.0], [23....",0,1
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Neutral,"[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[4.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",0,1


In [39]:
df6['dums'] = df6[['emo_happy','emo_unhappy']].apply(lambda x: pd.Series([x.values]), axis=1)

In [40]:
df6.to_csv(pth2/'df_fer_top.csv')

In [43]:
df6.shape

(35887, 9)

In [42]:
df6.head()

,pixels,emotion_names,pixar1,pixar2,emo_arr1,emo_arr,emo_happy,emo_unhappy,dums
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Angry,"[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63....","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[[70.0], [80.0], [82.0], [72.0], [58.0], [58....",0,1,"[0, 1]"
1,151 150 147 155 148 133 111 140 170 174 182 15...,Angry,"[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 111...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[[151.0], [150.0], [147.0], [155.0], [148.0],...",0,1,"[0, 1]"
2,231 212 156 164 174 138 161 173 182 200 106 38...,Fear,"[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 161...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16...","[[[231.0], [212.0], [156.0], [164.0], [174.0],...",0,1,"[0, 1]"
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Sad,"[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20....","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20...","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20...","[[[24.0], [32.0], [36.0], [30.0], [32.0], [23....",0,1,"[0, 1]"
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Neutral,"[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[[4.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0...",0,1,"[0, 1]"


In [45]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   pixels         35887 non-null  object
 1   emotion_names  35887 non-null  object
 2   pixar1         35887 non-null  object
 3   pixar2         35887 non-null  object
 4   emo_arr1       35887 non-null  object
 5   emo_arr        35887 non-null  object
 6   emo_happy      35887 non-null  uint8 
 7   emo_unhappy    35887 non-null  uint8 
 8   dums           35887 non-null  object
dtypes: object(7), uint8(2)
memory usage: 2.0+ MB


In [50]:
type(df6.pixar1[0])

list

In [54]:
df6.emo_arr[0].shape

(48, 48, 1)

# EL wueno es emo_arr

In [56]:
X = (np.stack(df6['emo_arr'])) / 255.0
y = np.stack(df6.dums)
X.shape, y.shape

((35887, 48, 48, 1), (35887, 2))

In [59]:
X_train, X_testval, y_train, y_testval = train_test_split(X,y,test_size=0.2)

In [60]:
X_test, X_val, y_test, y_val = train_test_split(X_testval,y_testval,test_size=0.5)

In [61]:
X_train.shape

(28709, 48, 48, 1)

In [62]:
y_train.shape

(28709, 2)

In [63]:
X_test.shape

(3589, 48, 48, 1)

In [64]:
y_test.shape

(3589, 2)

In [65]:
X_val.shape

(3589, 48, 48, 1)

In [66]:
y_val.shape

(3589, 2)

In [67]:
X_test.shape, X_val.shape, y_test.shape, y_val.shape 

((3589, 48, 48, 1), (3589, 48, 48, 1), (3589, 2), (3589, 2))

In [ ]:
def transfImag(path, new_path):
    
    # ROCKET
    """
    recibe carpeta, en cada foto de esa carpeta:
    lectura
    to gray
    facecascade
    por xywh en cada cara:
        array
        reshape array a 2d
        stack array 3d
        normalizar
        expand 4d
    devuelve array x cada foto para pasarselo al modelo
    """
    counter_imgs = 0
    counter_faces = 0
    
    X_ = pd.Series([], dtype='float64')
    
    for file in sorted(path.iterdir()):
        
        counter_imgs += 1

        input_img1 = cv2.imread(str(file))
        input_img2 = cv2.cvtColor(input_img1, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(input_img2, 1.25, 6)
        
        for (x,y,w,h) in faces:
            
            counter_faces += 1
            img_data1 = input_img2 [y:y+h,x:x+w]
            img_data2 = cv2.resize (img_data1,(48,48))

            img_data3 = np.stack(img_data2) 
            img_data4 = img_data2 / 255.0
            img_data5 = img_data3 / 255.0
            
            img_data6 = np.expand_dims(img_data5,axis=0).reshape(np.expand_dims(img_data5,axis=0).shape[0], 48, 48, 1)
            
            print(img_data6.shape)
            print(img_data6)
            
            img_datashow = img_data3*255
            img_show = Image.fromarray(img_datashow)
            file_to_save = file.name.replace(".",f"_face{counter_faces}.")
            img_show.save(str(new_path/file_to_save))
            
            counter_faces = 0
            
            arr_for_model = img_data6
            
            
            return arr_for_model
    
    print('YAAA!')    
    return counter_imgs
        
    

In [ ]:
dfplease.head()

In [ ]:
def base_model():
    model = Sequential()
    input_shape = (48,48,1)
    #1st convolution layer
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    #2nd convolution layer
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    #3rd convolution layer
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))

    my_optimiser = tf.keras.optimizers.Adam(
    learning_rate = 0.001, beta_1=0.9, beta_2=0.999, 
        epsilon=1e-07, amsgrad=False, name='Adam')
    
    model.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=my_optimiser)
    
    return model

In [ ]:
model_1 = base_model()

model_1.fit(X1_train, y1_train, 
            validation_data=(X1_val, y1_val), 
            epochs=15,
            verbose=1, 
            batch_size=50)

model_1.summary()

In [ ]:
impotd

In [ ]:
#transfImag(demo_imgs, demo_imgs_faces)

In [ ]:
# HAAR CASCADE CLASSIFIER
def detect_face_eyes_smile(pth,new_pth):
    """
    Extracts all .jpg files from local path, 
    calls on haar cascade classifiers (frontalface, eyes and smile) 
    and draws detection rectangles on each .jpg 
    
    Takes: local path of directory with .jpg images
    
    Returns: individual windows for .jpg files with detection rectangles for face, eyes and smile
    """

    counter_imgs = 0
    counter_faces = 0
    counter_smiles = 0
    counter_eyes = 0
    face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
    smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')
    
    for file in sorted(pth.iterdir()):
        if file.suffix != '.jpg':
            pass
        else:
            counter_imgs += 1
            print(file.name)
            
            img = cv2.imread(str(file))
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            #plt.imshow(img)

            # FRONTAL FACE 
            
            faces = face_cascade.detectMultiScale(
                img_gray, 
                scaleFactor=1.06,                
                minNeighbors=7,
                minSize=(30, 30), 
                flags=cv2.CASCADE_SCALE_IMAGE)
            if faces is None:
                print("No Face Found")

            for (fx,fy,fw,fh) in faces:
                
                counter_faces += 1
                roi_gray = img_gray [fy:fy+fh, fx:fx+fw] # region of interest for detection
                roi_gray2 = cv2.resize (roi_gray, (48,48))
                roi_gray3 = np.stack(roi_gray2)
                roi_gray4 = roi_gray2 / 255
                
                roi_color = img[fy:fy+fh, fx:fx+fw] # region of interest for mapping rectangle
                roi_color2 = cv2.resize (roi_color, (48,48))
                roi_color3 = np.stack(roi_color2)
                roi_color4 = roi_color2 / 255
                
                cv2.rectangle(
                    img,
                    (fx,fy),
                    (fx+fw,fy+fh),
                    #(127,0,255),
                    (0,255,0),
                    2)

                # SMILES 

                smiles = smile_cascade.detectMultiScale(
                    roi_gray, 
                    scaleFactor = 1.35, 
                    minNeighbors = 8)

                for (sx, sy, sw, sh) in smiles:
                    counter_smiles += 1
                    cv2.rectangle(
                        roi_color,
                        (sx, sy),
                        (sx + sw, sy + sh),
                        #(255, 0, 130),
                        #(0,220,80),
                        (127,0,255),
                        1)

                # EYES

                eyes = eye_cascade.detectMultiScale(
                    roi_gray,
                    scaleFactor=1.05,
                    minNeighbors = 6)

                for (ex,ey,ew,eh) in eyes:
                    counter_eyes += 1
                    cv2.rectangle(
                        roi_color, 
                        (ex , ey),
                        (ex + ew, ey + eh),
                        (0,255,255),
                        1)
            
                # save images with detected regions
                file_to_save = file.name.replace(".",f"_face{counter_faces}.")
                
                #cv2.imwrite(str(pth.parent/'demo_faces'/file_to_save),img)
                cv2.imwrite(str(new_pth/file_to_save),roi_color)
                counter_imgs = 0
                counter_faces = 0
            # show the output frame
            cv2.imshow(f"img{file_to_save}", img)
            key = cv2.waitKey(100) & 0xFF
            
    cv2.destroyAllWindows(f"img{file_to_save}")
    

            
            

    
    """
        # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                # do a bit of cleanup
                cv2.destroyAllWindows()
                break
        
    # do a bit of cleanup
    cv2.destroyAllWindows()
cv2.destroyAllWindows()


        """



In [ ]:
demo_imgs = Path.cwd().parent/'demo'
demo_imgs_faces = Path.cwd().parent/demo2
demo1 = demo_imgs/'A_0.jpg'
demo2 = demo_imgs/'demooo_01.jpg'

In [ ]:
face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')

In [ ]:
cv2.imwrite(str(pth.parent/*'_haar'/file.name),roi_color)

In [ ]:
str(pth.parent/'*_haar'/file.name)